In [1]:
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib  import colors
import load_functions
%matplotlib inline

In [2]:
smapsalinity = False

figdir = '/data_local/Satellite/Salinity/SMOS/figures'
salinityfile = '/data_local/Satellite/Salinity/SMOS/BEC_L4_SST_B_20160630T004729_20160709T005029_2013001_005_200.nc'

if smapsalinity:
    figdir = '/data_local/Satellite/Salinity/Smap/figures/'
    salinityfile = '/data_local/Satellite/SSS_Smap/sss_smap_8day_running_2016_182_v1.0.nc'

# Create figure directory if neccesary
if not(os.path.exists(figdir)):
    os.makedirs(figdir)
    
figname = os.path.basename(salinityfile)[:-3]
figname = figname.replace('.', '_')
figname = os.path.join(figdir, figname)

In [3]:
figdir

'/data_local/Satellite/Salinity/SMOS/figures'

Options for the plot

In [4]:
cmap=plt.cm.RdYlBu_r

vmin = 34.
vmax = 40.
bounds = np.arange(vmin, vmax + 0.01, 5)
norm = colors.Normalize(vmin=vmin, vmax=vmax + 0.01)
levels2plot = np.arange(vmin, vmax, 0.05)

Prepare projection (global)

In [5]:
lonmap = 0.
latmap = 20.

# Create basemap then used for the plot
m = Basemap(projection='ortho', lat_0 = latmap, lon_0 = lonmap, 
            resolution = 'c')

# Load the salinity from the file

In [6]:
lon, lat, psal = load_functions.load_salinity_L4_SMOS(salinityfile)

Apply shift to the coordinates and mask bad values

In [7]:
if smapsalinity:
    lonindwest = np.where(lon > 180.)[0]
    lonindeast = np.where(lon <= 180.)[0]

    lonnew = np.hstack((lon[lonindwest] - 360., lon[lonindeast]))
    psalnew = np.hstack((psal[:, lonindwest], psal[:, lonindeast]))
    psalnew = np.ma.masked_less(psalnew, 0., copy=True)

# Make the plot

In [8]:
llon, llat  = np.meshgrid(lon + 0.125, lat+ 0.125)
x, y = m(llon, llat)

Sub-sampling of the data

In [10]:
MM = 1
fig = plt.figure()
ax = fig.add_subplot(111)
m.ax = ax

contour = m.contourf(x[::MM, ::MM], y[::MM, ::MM], psal[::MM, ::MM], 
                     levels2plot, cmap=cmap, norm=norm, extend='both')

m.bluemarble()
m.drawcountries(linewidth=0.25)
cb = plt.colorbar(contour)
cb.set_clim(vmin, vmax)
plt.savefig(figname, dpi=300, facecolor='w', edgecolor='w',
         transparent=False, bbox_inches='tight', pad_inches=0.1)
# plt.show()
plt.close()

In [11]:
figname

'/data_local/Satellite/Salinity/SMOS/figures/BEC_L4_SST_B_20160630T004729_20160709T005029_2013001_005_200'